In [ ]:
# prompt: import python required library for a data analysis
# import time

# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import RFE
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import StackingClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import LabelEncoder

In [1]:
import joblib
import os
import pandas as pd
import numpy as np
from tabulate import tabulate
import datetime

In [2]:
# df = pd.read_excel('SSL_T.xlsx')
# df = pd.read_excel('SSL_T_MR.xlsx')
# df = pd.read_excel('SSL_M.xlsx')

# df = pd.read_excel('Train.xlsx')
# df = pd.read_excel('Test.xlsx')
# df = pd.read_excel('Train_Test.xlsx')

pd.set_option('display.max_rows', None)

##Overall Ratio Function

In [3]:
def get_feature_data(p_state, index,row):
       # Retrieve stats for individual features
      row_data = {}
      time_15min = row['15Min']
      trade_count_15m, pl_15m, win_rate_15m = precomputed_stats[('15Min', time_15min)]
      row_data['15Min_Trade Count'] = trade_count_15m
      row_data['15Min_Win Rate'] = win_rate_15m
      row_data['15Min_Profit/Loss'] = pl_15m

      hour = row['Hour']
      trade_count_hour, pl_hour, win_rate_hour = precomputed_stats[('Hour', hour)]
      row_data['Hour_Trade Count'] = trade_count_hour
      row_data['Hour_Win Rate'] = win_rate_hour
      row_data['Hour_Profit/Loss'] = pl_hour

      weekday = row['Start_Weekday']
      trade_count_weekday, pl_weekday, win_rate_weekday = precomputed_stats[('Weekday', weekday)]
      row_data['Weekday_Trade Count'] = trade_count_weekday
      row_data['Weekday_Win Rate'] = win_rate_weekday
      row_data['Weekday_Profit/Loss'] = pl_weekday

      month = row['Month']
      trade_count_month, pl_month, win_rate_month = precomputed_stats[('Month', month)]
      row_data['Month_Trade Count'] = trade_count_month
      row_data['Month_Win Rate'] = win_rate_month
      row_data['Month_Profit/Loss'] = pl_month

      criteria = row['Criteria']
      trade_count_criteria, pl_criteria, win_rate_criteria = precomputed_stats[('Criteria', criteria)]
      row_data['Criteria_Trade Count'] = trade_count_criteria
      row_data['Criteria_Win Rate'] = win_rate_criteria
      row_data['Criteria_Profit/Loss'] = pl_criteria


      # Retrieve stats for combined features
      trade_count_weekday_15m, pl_weekday_15m, win_rate_weekday_15m = precomputed_stats[('Weekday_15Min', weekday, time_15min)]
      row_data['Weekday_15Min_Trade Count'] = trade_count_weekday_15m
      row_data['Weekday_15Min_Win Rate'] = win_rate_weekday_15m
      row_data['Weekday_15Min_Profit/Loss'] = pl_weekday_15m

      trade_count_weekday_hour, pl_weekday_hour, win_rate_weekday_hour = precomputed_stats[('Weekday_Hour', weekday, hour)]
      row_data['Weekday_Hour_Trade Count'] = trade_count_weekday_hour
      row_data['Weekday_Hour_Win Rate'] = win_rate_weekday_hour
      row_data['Weekday_Hour_Profit/Loss'] = pl_weekday_hour

      trade_count_weekday_month, pl_weekday_month, win_rate_weekday_month = precomputed_stats[('Weekday_Month', weekday, month)]
      row_data['Weekday_Month_Trade Count'] = trade_count_weekday_month
      row_data['Weekday_Month_Win Rate'] = win_rate_weekday_month
      row_data['Weekday_Month_Profit/Loss'] = pl_weekday_month

      trade_count_criteria_15m, pl_criteria_15m, win_rate_criteria_15m = precomputed_stats[('Criteria_15Min', criteria, time_15min)]
      row_data['Criteria_15Min_Trade Count'] = trade_count_criteria_15m
      row_data['Criteria_15Min_Win Rate'] = win_rate_criteria_15m
      row_data['Criteria_15Min_Profit/Loss'] = pl_criteria_15m

      trade_count_criteria_hour, pl_criteria_hour, win_rate_criteria_hour = precomputed_stats[('Criteria_Hour', criteria, hour)]
      row_data['Criteria_Hour_Trade Count'] = trade_count_criteria_hour
      row_data['Criteria_Hour_Win Rate'] = win_rate_criteria_hour
      row_data['Criteria_Hour_Profit/Loss'] = pl_criteria_hour

      trade_count_criteria_weekday, pl_criteria_weekday, win_rate_criteria_weekday = precomputed_stats[('Criteria_Weekday', criteria, weekday)]
      row_data['Criteria_Weekday_Trade Count'] = trade_count_criteria_weekday
      row_data['Criteria_Weekday_Win Rate'] = win_rate_criteria_weekday
      row_data['Criteria_Weekday_Profit/Loss'] = pl_criteria_weekday

      trade_count_criteria_month, pl_criteria_month, win_rate_criteria_month = precomputed_stats[('Criteria_Month', criteria, month)]
      row_data['Criteria_Month_Trade Count'] = trade_count_criteria_month
      row_data['Criteria_Month_Win Rate'] = win_rate_criteria_month
      row_data['Criteria_Month_Profit/Loss'] = pl_criteria_month

      trade_count_hour_month, pl_hour_month, win_rate_hour_month = precomputed_stats[('Hour_Month', hour, month)]
      row_data['Hour_Month_Trade Count'] = trade_count_hour_month
      row_data['Hour_Month_Win Rate'] = win_rate_hour_month
      row_data['Hour_Month_Profit/Loss'] = pl_hour_month

      trade_count_15min_month, pl_15min_month, win_rate_15min_month = precomputed_stats[('15Min_Month', time_15min, month)]
      row_data['15Min_Month_Trade Count'] = trade_count_15min_month
      row_data['15Min_Month_Win Rate'] = win_rate_15min_month
      row_data['15Min_Month_Profit/Loss'] = pl_15min_month

      return row_data


In [4]:
def show_model_result(model_name, accuracy, class_report):
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(class_report)

##Get Train Data From Input

In [5]:
# prompt: get_train_data_for_input(df, time_input, date_input, criteria_input): instead of this get_train_data_for_input_optimized(precompute_stats, time_input, date_input, criteria_input) so that it can take data from precompute state instead of df

def get_train_data_for_input(precomputed_stats, time_input, date_input, criteria_input, training_feature_cols=[]):

    # Convert inputs
    try:
        input_time = datetime.datetime.strptime(time_input, '%H:%M').time()
    except ValueError:
        print("Invalid time format. Use HH:MM.")
        return pd.DataFrame()

    try:
        input_date = datetime.datetime.strptime(date_input, '%m/%d/%Y')
    except ValueError:
        print("Invalid date format. Use M/D/YYYY.")
        return pd.DataFrame()

    row_data = {}
    input_datetime = datetime.datetime.combine(input_date, input_time)

    time_15min = input_datetime.strftime('%H:%M')
    hour = input_datetime.hour
    weekday = input_datetime.strftime('%A')
    month = input_datetime.strftime('%B')
    criteria = criteria_input

    # Retrieve stats from precomputed_stats dictionary
    try:
        row = {'15Min': time_15min, 'Hour': hour, 'Start_Weekday': weekday, 'Month': month, 'Criteria': criteria}
        row_data = get_feature_data(precomputed_stats, 0, row)

    except KeyError as e:
        print(f"Error: Missing precomputed statistic for key {e}. Ensure create_precomputed_stats covers all combinations you need.")
        return pd.DataFrame()


    # Create DataFrame from the calculated row_data
    input_features_df = pd.DataFrame([row_data])
    # Reindex the input_features_df to match the training columns, filling missing with 0
    input_features_df = input_features_df.reindex(columns=training_feature_cols, fill_value=0)

    # Convert all columns to the same data type as the training features (assuming int from previous steps)
    for col in input_features_df.columns:
        try:
             input_features_df[col] = pd.to_numeric(input_features_df[col], errors='coerce')
             input_features_df[col] = input_features_df[col].fillna(0).astype(int)
        except ValueError:
            print(f"Warning: Could not convert input column '{col}' to integer.")


    return input_features_df

# Example usage (assuming X_train is defined from the previous code):
# Get the column names from your training features DataFrame

# Call the optimized function
# input_data = get_train_data_for_input_optimized(precomputed_stats, '09:30', '01/8/2023', 'ELC', training_feature_cols)
# print(input_data)


##Show Prediciton

In [6]:
# --- Prediction using the trained model ---
def predict_trade_result(model, input_features_df):
    """
    Predicts the result of a trade based on the input features using the trained model.

    Args:
        model: The trained machine learning model.
        input_features_df (pd.DataFrame): A DataFrame with one row containing
                                          the features for prediction, matching
                                          the format of the training features.

    Returns:
        int: The predicted class (e.g., 1 for Win, 0 for Loss).
        float: The predicted probability of the positive class (Win).
    """
    if input_features_df.empty:
        print("Cannot predict: Invalid input features.")
        return None, None

    # Make prediction
    predicted_class = model.predict(input_features_df)[0]
    if predicted_class == 0:
        predicted_proba = model.predict_proba(input_features_df)
    else:
        predicted_proba = model.predict_proba(input_features_df)

    return predicted_class, predicted_proba

In [7]:
  # pip install tabulate if needed

def show_predictions(time_input, date_input, criteria_input, CW, CL, training_feature_cols, model_names, model_keys,b_models=None, m_models=None, b_acc=None, m_acc=None, Max_R=False, w_threshold=2):
    mr = "_max_r" if Max_R else ""
    # Initialize models and their keys
    binary_models = b_models if b_models is not None else [globals()[f"{k}_model{mr}"] for k in model_keys]
    binary_accuracies = b_acc if b_acc is not None else [globals()[f"{k}_accuracy{mr}"] for k in model_keys]

    multiclass_models = m_models if m_models is not None else [globals()[f"{k}_model_m{mr}"] for k in model_keys]
    multiclass_accuracies = m_acc if m_acc is not None else [globals()[f"{k}_accuracy_m{mr}"] for k in model_keys]

    input_features = get_train_data_for_input(precomputed_stats, time_input, date_input, criteria_input, training_feature_cols)
    input_features['CL'] = [CL]
    input_features['CW'] = [CW]
    predictions = []
    predictions_multi = []
    if not input_features.empty:
        table_data = []

        for i, name in enumerate(model_names):
            # Predict binary
            pred_bin, prob_bin = predict_trade_result(binary_models[i], input_features)
            label_bin = 'Win' if pred_bin == 1 else 'Loss'
            acc_bin = f"{binary_accuracies[i]:.2f}"
            bin_str = f"{label_bin} ({np.max(prob_bin):.2f}), Acc: {acc_bin}"

            # Predict multiclass
            pred_multi, prob_multi = predict_trade_result(multiclass_models[i], input_features)
            acc_multi = f"{multiclass_accuracies[i]:.2f}"
            multi_str = f"Class {label_encoder.inverse_transform([pred_multi])[0]} ({np.max(prob_multi):.2f}), Acc: {acc_multi}"
            table_data.append([name, bin_str, multi_str])
            predictions.append(pred_bin)
            predictions_multi.append(pred_multi)
        predicted_win = predictions.count(1)
        predicted_loss = predictions.count(0)
        predicted_win_multi = len(predictions)-predictions_multi.count(0)
        predicted_loss_multi = predictions_multi.count(0)
        trade_decision = "Win" if predicted_win > w_threshold else "Loss"
        trade_decision_multi = "Win" if predicted_win_multi >= w_threshold else "Loss"
        max_r = "" if trade_decision_multi == "Loss" else "- Max R: "+str(np.max(predictions_multi))
        avg_r = "" if trade_decision_multi == "Loss" else "- Average R:"+str(np.mean([x for x in predictions_multi if x>0]))
        # Display table
        print(f"\n--- 📊 Combined Model Predictions for {date_input} {time_input} ({criteria_input}) -{mr} ---\n")
        print(f"Overall decision: ---- Binary: ** {trade_decision} **  Multi: ** {trade_decision_multi}** ---- {max_r} {avg_r}")
        print(f"Binary Models: Win Predicted **{predicted_win} time **    Multi Models: Win Predicted **{predicted_win_multi}**")
        print(tabulate(table_data, headers=["Model", "📘 Binary (Win/Loss)", "📗 Multi-Class (R Bucket)"], tablefmt="fancy_grid"))

    else:
        print("❌ Prediction could not be made due to invalid inputs or data.")


#Read Require Data

##Read Models

##Read From Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [29]:
# prompt: remove SMC directory from my files
%cd /content/
!rm -rf SMC/

/content


In [9]:
# Copy SMC folder from Drive to current working directory (optional)
!cp -r /content/drive/MyDrive/SMC /content/

# Move into the project directory
%cd /content/SMC

# List contents to confirm
!ls

cp: cannot stat '/content/drive/MyDrive/SMC': No such file or directory
[Errno 2] No such file or directory: '/content/SMC'
/content
sample_data


##Read From Git

In [10]:
# Clone the GitHub repository
!git clone https://github.com/MIARD/SMC.git

# Move into the cloned directory
%cd SMC

# List files to confirm
!ls

Cloning into 'SMC'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 31 (delta 16), reused 31 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 6.73 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/content/SMC
max_r_model_accuracies.xlsx    saved_models
precomputed_stats.pkl	       SSL_T_MR.xlsx
README.md		       SSL_T.xlsx
regular_model_accuracies.xlsx  training_feature_cols.pkl


##Read info

In [8]:
# prompt: read both accuracies from save xlxs and show their tables

# Read the saved accuracy dataframes
try:
  precomputed_stats = joblib.load('precomputed_stats.pkl')
  training_feature_cols = joblib.load('training_feature_cols.pkl')
  regular_accuracies = pd.read_excel("regular_model_accuracies.xlsx")
  max_r_accuracies = pd.read_excel("max_r_model_accuracies.xlsx")

  print("\n--- Regular Model Accuracies ---")
  print(tabulate(regular_accuracies, headers='keys', tablefmt='psql'))

  print("\n--- Max R Model Accuracies ---")
  print(tabulate(max_r_accuracies, headers='keys', tablefmt='psql'))

except FileNotFoundError:
  print("Accuracy files not found. Please run the model training section first.")
except Exception as e:
  print(f"An error occurred while reading the accuracy files: {e}")


--- Regular Model Accuracies ---
+----+---------------------+-------------------+-----------------------------------+
|    | Model               |   Binary Accuracy |   Multi-Class (R Bucket) Accuracy |
|----+---------------------+-------------------+-----------------------------------|
|  0 | RandomForest        |          0.878689 |                          0.790164 |
|  1 | XGBoost             |          0.852459 |                          0.796721 |
|  2 | Logistic Regression |          0.898361 |                          0.829508 |
|  3 | Voting Classifier   |          0.885246 |                          0.819672 |
|  4 | Stacking Classifier |          0.898361 |                          0.829508 |
+----+---------------------+-------------------+-----------------------------------+

--- Max R Model Accuracies ---
+----+---------------------------+-------------------+-----------------------------------+
|    | Model                     |   Binary Accuracy |   Multi-Class (R Bucket

##Loading Models

In [11]:
# prompt: read all 20 models separately  regular binary, regular multi, max r binary, max r multi, . so that i can pass binary model and multi models list to function easily

# Define a directory to save the models
model_dir = 'saved_models'

# Define lists to store the loaded models
regular_binary_models = []
regular_multi_models = []
max_r_binary_models = []
max_r_multi_models = []

# List of model base names (keys used in filenames)
model_keys = ['RF', 'XVG', 'LR', 'VC', 'S']

# Load regular binary models
print("Loading regular binary models...")
for key in model_keys:
    filename = os.path.join(model_dir, f'{key}_model.pkl')
    try:
        model = joblib.load(filename)
        regular_binary_models.append(model)
        print(f"Loaded {key}_model")
    except FileNotFoundError:
        print(f"Error: {key}_model.pkl not found. Please run the saving section first.")
    except Exception as e:
        print(f"Error loading {key}_model: {e}")

# Load regular multi-class models
print("\nLoading regular multi-class models...")
for key in model_keys:
    filename = os.path.join(model_dir, f'{key}_model_m.pkl')
    try:
        model = joblib.load(filename)
        regular_multi_models.append(model)
        print(f"Loaded {key}_model_m")
    except FileNotFoundError:
        print(f"Error: {key}_model_m.pkl not found. Please run the saving section first.")
    except Exception as e:
        print(f"Error loading {key}_model_m: {e}")

# Load max R binary models
print("\nLoading Max R binary models...")
for key in model_keys:
    filename = os.path.join(model_dir, f'{key}_model_max_r.pkl')
    try:
        model = joblib.load(filename)
        max_r_binary_models.append(model)
        print(f"Loaded {key}_model_max_r")
    except FileNotFoundError:
        print(f"Error: {key}_model_max_r.pkl not found. Please run the saving section first.")
    except Exception as e:
        print(f"Error loading {key}_model_max_r: {e}")

# Load max R multi-class models
print("\nLoading Max R multi-class models...")
for key in model_keys:
    filename = os.path.join(model_dir, f'{key}_model_m_max_r.pkl')
    try:
        model = joblib.load(filename)
        max_r_multi_models.append(model)
        print(f"Loaded {key}_model_m_max_r")
    except FileNotFoundError:
        print(f"Error: {key}_model_m_max_r.pkl not found. Please run the saving section first.")
    except Exception as e:
        print(f"Error loading {key}_model_m_max_r: {e}")

# Load Label Encoders
print("\nLoading Label Encoders...")
try:
    label_encoder = joblib.load(os.path.join(model_dir, 'label_encoder.pkl'))
    print("Loaded label_encoder")
except FileNotFoundError:
    print("Error: label_encoder.pkl not found.")
except Exception as e:
    print(f"Error loading label_encoder: {e}")

try:
    label_encoder_max_r = joblib.load(os.path.join(model_dir, 'label_encoder_max_r.pkl'))
    print("Loaded label_encoder_max_r")
except FileNotFoundError:
    print("Error: label_encoder_max_r.pkl not found.")
except Exception as e:
    print(f"Error loading label_encoder_max_r: {e}")


# Now you have the models loaded into these lists:
# regular_binary_models
# regular_multi_models
# max_r_binary_models
# max_r_multi_models

# You can pass these lists to other functions as needed.
# For example, you could create a function that takes a list of models and a dataframe
# and performs predictions or evaluation.


Loading regular binary models...
Loaded RF_model
Loaded XVG_model
Loaded LR_model


c:\Users\HP\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:36:24] WARNING: C:\b\abs_90_bwj_86a\croot\xgboost-split_1724073762025\work\src\gbm\../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


Loaded VC_model
Loaded S_model

Loading regular multi-class models...
Loaded RF_model_m
Loaded XVG_model_m
Loaded LR_model_m
Loaded VC_model_m
Loaded S_model_m

Loading Max R binary models...
Loaded RF_model_max_r
Loaded XVG_model_max_r
Loaded LR_model_max_r
Loaded VC_model_max_r
Loaded S_model_max_r

Loading Max R multi-class models...
Loaded RF_model_m_max_r
Loaded XVG_model_m_max_r
Loaded LR_model_m_max_r
Loaded VC_model_m_max_r
Loaded S_model_m_max_r

Loading Label Encoders...
Loaded label_encoder
Loaded label_encoder_max_r


##Prediction App

In [34]:
import datetime
print(datetime.datetime.now(datetime.UTC).strftime('%m/%d/%Y'))
print(datetime.datetime.now(datetime.UTC).strftime('%H:%M'))


06/29/2025
15:39


In [39]:
# time_input = "15:00"  #@param {type:"string"}
# date_input = "6/29/2025"  #@param {type:"string"}
# criteria_input = "LG RGC"  #@param {type:"string"}
# CW = 0 #@param {type:"integer"}
# CL = 3 #@param {type:"integer"}

time_input = "15:45"  #@param {type:"string"}
date_input = datetime.datetime.now(datetime.UTC).strftime('%m/%d/%Y')  #@param {type:"string"}
criteria_input = "ELC"  #@param {type:"string"}
CW = 0 #@param {type:"integer"}
CL = 3 #@param {type:"integer"}

model_names = ['Random Forest', 'XGBoost', 'Logistic Regression', 'Voting Classifier', 'Stacking Classifier']
model_names_max_r = ['Random Forest', 'XGBoost', 'Logistic Regression', 'Voting Classifier', 'Stacking Classifier']
model_keys = ['RF', 'XVG', 'LR', 'VC', 'S']

show_predictions(time_input, date_input, criteria_input, CW, CL,training_feature_cols, model_names, model_keys, b_models=regular_binary_models, m_models=regular_multi_models, b_acc=regular_accuracies['Binary Accuracy'], m_acc=regular_accuracies['Multi-Class (R Bucket) Accuracy'])



show_predictions(time_input, date_input, criteria_input, CW, CL,training_feature_cols, model_names_max_r, model_keys, b_models=max_r_binary_models, m_models=max_r_multi_models, b_acc=max_r_accuracies['Binary Accuracy'], m_acc=max_r_accuracies['Multi-Class (R Bucket) Accuracy'], Max_R=True)
# List of predictions from the trained models



AttributeError: module 'datetime' has no attribute 'strptime'

In [35]:
import gradio as gr

def predict_trading(time_input, date_input, criteria_input, CW, CL):
    # Your existing code: preprocess, predict, return result text
    result_text = show_predictions(time_input, date_input, criteria_input, CW, CL,
                                   training_feature_cols, model_names, model_keys,
                                   b_models=regular_binary_models, m_models=regular_multi_models,
                                   b_acc=regular_accuracies['Binary Accuracy'],
                                   m_acc=regular_accuracies['Multi-Class (R Bucket) Accuracy'])
    return result_text

app = gr.Interface(
    fn=predict_trading,
    inputs=[
        gr.Textbox(label="Time"),
        gr.Textbox(label="Date"),
        gr.Textbox(label="Criteria"),
        gr.Number(label="CW"),
        gr.Number(label="CL")
    ],
    outputs="text"
)

app.launch(share=True) 

ModuleNotFoundError: No module named 'pyaudioop'